<a href="https://colab.research.google.com/github/jokercsi/Investment-Portfolio-Optimization/blob/main/NewsScarpingTitle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# 필요한 라이브러리 가져오기
from bs4 import BeautifulSoup
import requests
import re

In [53]:
# 대상 사이트 : investing.com
url = "https://www.investing.com/news/stock-market-news"

# 프로토콜 status 확인
# 200이면 정상
resp = requests.get(url)
print(resp)

<Response [200]>


In [54]:
import pandas as pd

# csv 만들리 위해 DataFrame 사용하기
sample = [('sampleTitle','sampleDate','samplePublisher')]
df = pd.DataFrame(sample, columns=['Title','Date','Publisher'])

In [55]:
# string date to datetime
# stirng 날짜를 datetime으로 바꾸기 
# 현재 데이터 값 예시: Sep 15, 2022 
from datetime import datetime

In [56]:
# 링크가 주어지면, 기사 제목과 본문을 크롤링하는 코드
def get_news_by_url(url):

    # 각자 본인의 User-Agent 정보 입력 
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    res = requests.get(url, headers = headers)
    bs = BeautifulSoup(res.content, 'html.parser')

    news_list = []

    # 뉴스 기사 가져오기 
    # Article 태그의 js-article-item클래스만 가져오기 
    for i in bs.find('section',id="leftColumn").find_all('article', "js-article-item"):
      # i['class'] => i의 클래스 이름 가져오기
      # 스폰서 광고 기사는 없애기
      if "sponsoredArticle" in i['class']:
        continue
        
      else:
        # 댓글이 없으면 무시
        if i.find('a', 'js-scrollto-comment') == None:
          continue

        # 댓글이 있으면
        else:
          # 댓글수가 5보다 많은 기사만 가져오기
          if int(i.find('a', 'js-scrollto-comment').text) > 5:

            #출판사 구하기
            span = i.find('span', 'articleDetails').text
            x = span.split("-")
            publisher = x[0][3:]
            #쓸데 없는 빈칸 없애기
            publisher = publisher[:-1]

            #기사 제목 구하기
            title = i.find('a', 'title').text

            #기사 날짜 구하기
            #Sep 15, 2022 
            dateOutput = i.find('span', 'date').text
            date = dateOutput[3:]
            date = datetime.strptime(date, '%b %d, %Y')

            news = [title, date, publisher]

            news_list.append(news)

    return news_list

In [57]:
for i in range(5,10):
  url = 'https://www.investing.com/news/stock-market-news/' + str(i)

  list = get_news_by_url(url)

  # output이 존재한다면 (없을수도있으니까)
  if list:
    print("Yes, we have data in", i)
    for j in list:
      # 행 추가하기 
      df = df.append({'Title': j[0], 'Date':j[1], 'Publisher':j[2] }, ignore_index=True)


Yes, we have data in 5
Yes, we have data in 6
Yes, we have data in 7
Yes, we have data in 9


In [58]:
df

,Title,Date,Publisher
0,sampleTitle,sampleDate,samplePublisher
1,"U.S. rail strike averted, but labor deal faces...",2022-09-15 00:00:00,Reuters
2,"Wall St tumbles amid Fed tightening jitters, e...",2022-09-15 00:00:00,Reuters
3,Elon Musk accuses Twitter of security lapses i...,2022-09-15 00:00:00,Reuters
4,FedEx Issues Massive Profit Warning as Global ...,2022-09-15 00:00:00,Investing.com
5,Stock Market Today: Dow Stumbles as Fears of L...,2022-09-15 00:00:00,Investing.com
6,"Michigan Sentiment, EV Showcase, Movie Premier...",2022-09-15 00:00:00,Investing.com
7,President Biden Announces $900M for EV Chargin...,2022-09-15 00:00:00,Investing.com
8,U.S. Stocks Wobble After Strong Reports on Ret...,2022-09-15 00:00:00,Investing.com


In [60]:
# csv파일로 저장하기
df.to_csv('investingcom.csv', sep=',')